# 初始化

In [1]:
import os
import gc
import cv2
import time
import numpy as np  
np.set_printoptions(threshold=np.inf)
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

from keras import layers
from keras.models import Model
from keras.layers import Input,Dense,Flatten,Dropout,ZeroPadding2D,BatchNormalization,Activation,add,AveragePooling2D
from keras.layers.convolutional import Conv2D,MaxPooling2D  
from keras.applications.imagenet_utils import preprocess_input

Using TensorFlow backend.


# 读取训练数据

In [2]:
dataset = 'UCM'
base_path='./Dataset/' + dataset
ModelName = "SeResNet50"
batch_size = 16
epochs = 50

X_Train = np.load(base_path + "/X_Train_224.npy");
Y_Train = np.load(base_path + "/Y_Train.npy");

print("Get %d Train Images" %(len(X_Train)))    
num_classes = Y_Train.shape[1]

Get 26880 Train Images


# 原生ResNet-50

In [3]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x 
    
from keras.optimizers import SGD  
def ResNet(shape):
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
        
    img_input = Input(shape=shape)  

    x = ZeroPadding2D((3, 3))(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x = layers.pooling.GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(1024, activation='relu', name='fc1024')(x)  
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu', name='fc512')(x)  
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax', name='fc')(x)  

    model = Model(inputs=img_input,outputs=x, name="ResNet50")  
    sgd = SGD(decay=0.0001,momentum=0.9)  
    model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])  
    
    return model

In [4]:
model = ResNet(X_Train.shape[1:])
model.load_weights('./models/ResNet50.h5', by_name=True)
model.summary()
#OutPutLayer = model.get_layer('flatten_1').output
#prediction = Dense(num_classes, activation='softmax')(OutPutLayer)
#new_model = Model(inputs=model.input, outputs=prediction, name="ResNet50")
#sgd = SGD(decay=0.0001,momentum=0.9)  
#new_model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])  
#new_model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [5]:
history = model.fit(X_Train, Y_Train, batch_size=batch_size, epochs=epochs, shuffle=True)

Epoch 1/50
26880/26880 [==============================] - 183s 7ms/step - loss: 0.7855 - accuracy: 0.7747
Epoch 2/50
26880/26880 [==============================] - 178s 7ms/step - loss: 0.1369 - accuracy: 0.9602
Epoch 3/50
26880/26880 [==============================] - 178s 7ms/step - loss: 0.0545 - accuracy: 0.9842
Epoch 4/50
26880/26880 [==============================] - 178s 7ms/step - loss: 0.0347 - accuracy: 0.9910
Epoch 5/50
26880/26880 [==============================] - 180s 7ms/step - loss: 0.0226 - accuracy: 0.9932
Epoch 6/50
26880/26880 [==============================] - 182s 7ms/step - loss: 0.0127 - accuracy: 0.9962
Epoch 7/50
26880/26880 [==============================] - 180s 7ms/step - loss: 0.0043 - accuracy: 0.9989
Epoch 8/50
26880/26880 [==============================] - 181s 7ms/step - loss: 0.0073 - accuracy: 0.9981
Epoch 9/50
26880/26880 [==============================] - 180s 7ms/step - loss: 0.0059 - accuracy: 0.9984
Epoch 10/50
26880/26880 [=====================

# 保存模型

In [6]:
save_folder = 'models/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + dataset
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# serialize model to JSON
import pickle
model_json = model.to_json()
with open(os.path.join(save_folder, ModelName + ".json"), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save(os.path.join(save_folder, ModelName + ".h5"))
#pickle.dump(history.history, open('history/UCMerced_LandUse/AlexNet.p','wb'))
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + "Saved Model to disk")

2020-05-05 21:49:55Saved Model to disk
